## Exported Slice Data
I converted [from HDF5](https://www.getdatajoy.com/learn/Read_and_Write_HDF5_from_Python) [to PNG](https://pythonhosted.org/pypng/ex.html) to vizualze the splice maps. I wanted to make sure I understood how they were stored.

In [1]:
import h5py as h5
import numpy as np
import png
img = [0,0,0,1]
out = 'map.png'
class imgurl:
    def __init__(self,im_):
        self.img = im_
    def __getitem__(self, i):
        return '%08d' % self.img[i]
    def url(self):
        return 'w='+self[3]+'/z='+self[2]+'/y='+self[1]+',x='+self[0]
img = imgurl(img);
fi = h5.File('../temp/ids/tiles/'+img.url()+'.hdf5', 'r')
np.set_printoptions(threshold=np.nan)
data = np.array(fi.get('dataset_1'))*2
w = png.Writer(len(data),len(data),greyscale=1,bitdepth=16)
fo = open(out, 'wb')
w.write(fo, data)
fo.close()

## Stepped through Python
To see how each class sends content, I set breakpoints for the request types. I then print the file name at each request.
```
ipython
%run -d dojo.py ../mojo ../temp 1337 anything
b 173;; b 176;; c
[open browser]
n;; p r.request.uri;; c
```

To see how the websocket connection works, I set breakpoints at websocket.js:39 and websocket.js:45 as well as:
```
%run -d dojo.py ../mojo ../temp 1337 anything
b _dojo/websockets.py:39;; b _dojo/websockets.py:45;; c
[open browser]
```
#### subclasses:
- Datasource
    - Segmentation
    - Image
- Controller
- Database

## Stepped through Javascript
- Image and Segmentation data are handled differently in the Datasource because **Datasource.__query** values differ

|Function|Called from|
|-|-:|
|DOJO.init:|index.html:361|
|DOJO.viewer.init|**dojo.js:23**|
|loader.load_json|**viewer.js:83**|
|xhr.open:'/image/contents'|loader.js:19|
|*ServerLogic.handle*|**xhr**|:=img[0]|
|*Datasource.handle*|*image.py:83*|:=img[1]|
|**callback[viewer.js:83]**|*datasource.py:255*|:=img[2]|
|||
|loader.load_json|**viewer.js:103**|
|xhr.open:'/segmentation/contents'|loader.js:19|
|*ServerLogic.handle*|**xhr**|:=seg[0]|
|*Datasource.handle*|*segmentation.py:230*|:=seg[1]|
|**callback[viewer.js:103]**|*datasource.py:255*|:=seg[2]|
|||
|loader.load_json|**viewer.js:108**|
|xhr.open:'/segmentation/colormap'|loader.js:19|
|**callback[viewer.js:108]**|**xhr**|=seg[ : ]|
|||
|viewer.render|viewer.js:128|
|websocket.connect|viewer.js:131|
|**callback[dojo.js:23]**|viewer.js:133|
|||
|*Websockets.open*|websocket.js:18|
|*Controller.handshake*|*websockets.py:28*|
|*Websockets.send*|*controller.py:71*|
|**callback[websocket.js:36]**|*websockets.py:45*|
|||
|controller.receive|websocket.js:39|
|controller.send|controller.js:141|
|websocket.send|controller.js:529|
|*Websockets.on_message*|websocket.js:45|
|*Controller.on_message*|*websockets.py:39*|

- Start
    - *Controller.on_message*:212 sends merge table to client
    -  controller.receive:121

- Merge
    - controller.end_draw_merge:1494
    - For each link to merge
        - controller.merge: ?
    - controller.end_draw_merge:1526
    - controller.send_merge_table_subset:763
    - *Controller.on_message*: 240
    - *Controller.send_merge_table_subset*:144
    - controller.receive:148
        - TODO
    
- Save
    - controller.save:251
         - TODO
 
 ```
%run -d dojo.py ../mojo ../temp 1337 anything
b _dojo/controller.py:235;; c
[open browser]
```


- Master controller only has a merge function
- Performance has start_merge, merge, draw_merge, and end_draw_merge functions

#### 2016-03-07
- Applying smaller merge table on each merge
    - create small pair list for gl_merge_table
    - fragment shade for small pair list

#### 2016-03-13

Merge table loaded from database 
```
b _dojo/datasource.py:123
b _dojo/database.py:115
```

```
python -m pdb dojo.py ../mojo ../temp 1337 anything --nothreading --noreload
```